In [1]:
from selenium.webdriver import Chrome
import pandas as pd
import time
import os

url = "https://www.cathaybk.com.tw/cathaybk/personal/credit-card/cards/intro/list/"

# 開啟並取得國泰信用卡網頁。
driver = Chrome("./chromedriver.exe")
# 注意是否要放大，或自訂螢幕大小。
driver.maximize_window()
time.sleep(1)
driver.get(url)

In [2]:
#每個迴圈都要停一下 確定指令運作已完成,不然有時候跑太快 頁面還沒捲下去會找不到id
#確定你要的id有在畫面上,不然原始碼可能不會載入


#抓取每一個分頁的卡片名稱
title = []
for page in range(4):
    print("page=",page)
    time.sleep(1)
    js="var action=document.documentElement.scrollTop=10000"
    time.sleep(1)
    driver.execute_script(js)
    time.sleep(1)
    driver.find_element_by_id("layout_0_rightcontent_0_updatepanel_1_RptPagination_HlkPageNumber_"+str(page)).click()
    time.sleep(1)
    print("我在第",page+1,"頁")
    ps = driver.find_elements_by_class_name("card-name")
    for p in ps:
        name = p.text
        title.append(name)
    
#回到第一頁
driver.find_element_by_id("layout_0_rightcontent_0_updatepanel_1_RptPagination_HlkPageNumber_0").click()

page= 0
我在第 1 頁
page= 1
我在第 2 頁
page= 2
我在第 3 頁
page= 3
我在第 4 頁


In [3]:
#整理list內容,把為空的部分去除掉
#並且把字之間的空白去掉
title = [x for x in title if x != '']
title2 = []
for y in title:
    y = y.replace(" ", "")
    y = y.replace("(COMBO)", "")
    title2.append(y)
title2 = title2[:23]
print(title2)
print(len(title2))

['世界卡', '蝦皮購物聯名卡', 'KOKOicash聯名卡', 'KOKOCOMBOicash聯名卡', '亞洲萬里通聯名卡', '長榮航空聯名卡', '台塑聯名卡', 'Costco聯名卡', '享購鈦金卡', '現金回饋御璽卡', 'KOKO悠遊聯名卡', '鈦金商務卡', '尊榮御璽卡卡', '白金卡', '雙幣卡', 'eTag聯名卡', '遠東SOGO(悠遊)聯名卡', '太平洋SOGO百貨聯名卡', '國泰人壽聯名卡', '商務卡', 'KOKO簽帳金融卡', 'ｉ刷悠遊Debit卡', 'ｉ刷金融卡']
23


In [4]:
#滾輪往下滾
def scrolltop(pos):
    js = "document.documentElement.scrollTop=%s" % pos
    driver.execute_script(js)
    
#滾輪滾到底
def scrolldown():
    js = "document.documentElement.scrollTop=10000"
    driver.execute_script(js)
    
#去到下一頁
def backpage(page):
    driver.find_element_by_id(
        "layout_0_rightcontent_0_updatepanel_1_RptPagination_HlkPageNumber_{}".format(page-1)).click()
    # 給他五秒鐘的時間跑
    time.sleep(5)
    
#抓取網頁內的資料
context = []
def catch():
    temp = []
    #先判斷網頁有幾個上標
    title = []
    for num in range(10):
        try:
            lenth = driver.find_element_by_id("layout_0_rightcontent_1_liTab0"+str(num+1)).text
            #確保串列內字元不為空
            if lenth != "":
                title.append(lenth)
            else:
                continue
        except:
            break
    # 根據上標數做迴圈
    for item in range(len(title)):
        try:
            driver.find_element_by_id("layout_0_rightcontent_1_liTab0"+str(item + 1)).click()
            data = driver.find_element_by_id("layout_0_rightcontent_1_tab0"+str(item+1)+"_0_divParagraph")
            x = data.text
            
            #把每個上標的內容先存到temp裡面
            temp.append(x)
        except:
            #這裡要新增卡友權益
            continue
    sstr = ""
    sstr = sstr.join(temp)
    sstr = sstr.replace(" ","")
    sstr = sstr.replace("\r","")
    sstr = sstr.replace("\n","")
    context.append(sstr)
    # 上一頁到卡片清單頁
    for item2 in range(len(title)):
        driver.back()

In [5]:
# 起始位置設為200，注意螢幕大小，我的螢幕為14吋。
page = 1
while page < 5:
    try:
        pos = 200
        for i in range(10):
            pos += 220
            scrolltop(pos)
            # 注意抓取的值為list，要一個一個取出。
            driver.find_elements_by_xpath("//div[@class='card-features']/a[@class='link-learn-more']")[i].click()
            catch()
            print("現在到第幾頁:", page, "到第幾張卡片:", i + 1)
            time.sleep(1)
            if page == 1:
                driver.back()
                scrolltop(pos)
                time.sleep(1)
            else:
                driver.back()
                scrolldown()
                backpage(page)
        page += 1
        backpage(page)
    except:
        break
print("好像沒東西喔!!!")


現在到第幾頁: 1 到第幾張卡片: 1
現在到第幾頁: 1 到第幾張卡片: 2
現在到第幾頁: 1 到第幾張卡片: 3
現在到第幾頁: 1 到第幾張卡片: 4
現在到第幾頁: 1 到第幾張卡片: 5
現在到第幾頁: 1 到第幾張卡片: 6
現在到第幾頁: 1 到第幾張卡片: 7
現在到第幾頁: 1 到第幾張卡片: 8
現在到第幾頁: 1 到第幾張卡片: 9
現在到第幾頁: 1 到第幾張卡片: 10
現在到第幾頁: 2 到第幾張卡片: 1
現在到第幾頁: 2 到第幾張卡片: 2
現在到第幾頁: 2 到第幾張卡片: 3
現在到第幾頁: 2 到第幾張卡片: 4
現在到第幾頁: 2 到第幾張卡片: 5
現在到第幾頁: 2 到第幾張卡片: 6
現在到第幾頁: 2 到第幾張卡片: 7
現在到第幾頁: 2 到第幾張卡片: 8
現在到第幾頁: 2 到第幾張卡片: 9
現在到第幾頁: 2 到第幾張卡片: 10
現在到第幾頁: 3 到第幾張卡片: 1
現在到第幾頁: 3 到第幾張卡片: 2
現在到第幾頁: 3 到第幾張卡片: 3
現在到第幾頁: 3 到第幾張卡片: 4
現在到第幾頁: 3 到第幾張卡片: 5
現在到第幾頁: 3 到第幾張卡片: 6
好像沒東西喔!!!


In [6]:
context = context[:23]
print(context)
print(len(context))

['新卡滿額禮世界卡2019/7月-2019/10月新卡刷卡禮2019/7/1-2019/10/31期間首次申辦世界卡正卡，核卡日後60天內刷一般消費累積達門檻，享精選好禮回饋:<滿額禮>精緻滿額禮：達NT$15萬以上，「SamsonitePolygon25吋行李箱(藍)」乙個。尊榮滿額禮：達NT$60萬以上，「紅利積點30萬點」。<加碼禮>加碼禮：達NT$18萬以上，「兩次免費機場接或送禮遇」。贈品介紹精緻滿額禮：SamsonitePolygon25吋行李箱(藍)材質：100%PC(聚碳酸酯)尺寸：47X69X31cm淨重：4Kg容量：77L保修期間：3年(相關保修規範以原廠官網說明為準)商品詢問及維修電話:0800-088-349商品簡介：採用FLAT-MAX™結構設計「面蓋與主箱體為20:80比例」，行李箱20%是拉桿系統的面蓋，80%是主要儲存空間的主箱體，打造出特大容量。箱蓋側面有煞車止滑開關，選用日本Hinomoto的EASYBRAKE™煞車輪，拉桿兩側的輪子在煞車系統啟動時，仍可拖行。TSA海關鎖+隱藏式地址貼牌。尊榮滿額禮：紅利積點30萬點紅利積點30萬點可兌換高額航空里程/飯店積分、百貨禮券、商品，讓您輕鬆旅行、購物，盡享寫意自在品味生活。(瞭解更多「舞動紅利」紅利點數兌換辦法)紅利積點30萬點可兌換多樣化優惠：亞洲萬里通74,000里數，或長榮航空60,000哩，或LeClubAccorHotels雅高樂雅會獎勵積分20,000點新光三越百貨商品禮券17,000元，或遠東SOGO百貨商品券17,500元TreeMall紅利線上購物全站超過10萬種品項加碼禮：兩次免費機場接或送禮遇享指定區域至以下機場免費機場接或送禮遇2次。台北松山機場桃園國際機場台中國際機場高雄國際機場(小港機場)24小時服務預約專線(04)2206-1608(持卡人倘符合消費資格，本行將透過簡訊通知，持卡人收到簡訊後才可進行預約。)(本免費機場接送禮遇有限定服務區域，詳細服務適用條件與限制依官網公告為準。)注意事項生日貴賓尊寵禮遇Sisley生日獻禮專屬尊寵生日獻禮：貴賓本人憑世界卡與生日優惠券至全台Sisley專櫃接受專業肌膚諮詢並留下資料，即可兌換生日禮尊寵呵護組。頂級護膚服務現金折抵券NT$2,200：至Sisley指定專櫃或美膚中心預約療程，享NT$2,200療程折抵優

In [7]:
result = pd.DataFrame({
    "Title":title2,
    "context":context
})
result.to_csv("cathaybk.csv", encoding = "utf-8", index = False)
result

,Title,context
0,世界卡,新卡滿額禮世界卡2019/7月-2019/10月新卡刷卡禮2019/7/1-2019/10/...
1,蝦皮購物聯名卡,
2,KOKOicash聯名卡,新戶首刷禮享OPENPOINT2019/7/1~2019/12/31，新戶申辦KOKOica...
3,KOKOCOMBOicash聯名卡,申辦KOKOCOMBOicash聯名卡，新戶享KAKAOFRIENDS首刷禮3選1！108/...
4,亞洲萬里通聯名卡,里程加速器NT$15元=1亞洲萬里通聯名卡首創里程加速器，刷卡消費可直接累積「亞洲萬里通」里...
5,長榮航空聯名卡,啟動倍數引擎最優10元1哩長榮航空官網或國內各大旅行社刷國泰世華長榮航空聯名卡購買長榮/立榮...
6,台塑聯名卡,月月加油滿額登錄送108/10/1至108/12/31，持台塑聯名卡於台亞、福懋、西歐及標示...
7,Costco聯名卡,新戶首刷禮享500元多利金2019/9/1至2019/12/31(含)期間，成功申辦Cost...
8,享購鈦金卡,NEW!新戶全聯加碼禮全聯店內單筆刷滿NT$500贈NT$120現金回饋108/10/1~1...
9,現金回饋御璽卡,不限通路．最高2%現金回饋108/12/31前，無通路限制、無級距門檻、無回饋上限，刷越多回...


In [31]:
# print(context[8])

NEW!新戶全聯加碼禮全聯店內單筆刷滿NT$500贈NT$120現金回饋108/10/1~12/31於全聯通路(業務員推薦)或線上申辦進件之享購鈦金卡新戶，核卡後30天內於全聯店內刷卡消費(含PXPay交易，不含電子票證消費及自動加值交易)單筆滿NT$500(含以上)加贈現金回饋NT$120。再享新戶輕鬆刷好禮4選1活動»活動說明注意事項國外消費享1.7%現金回饋，國內消費最高1%現金回饋108/12/31前，國外刷卡消費享1.7%現金回饋，無級距門檻，無回饋上限，刷越多回饋越多!108/12/31前，國內刷卡消費享0.5%現金回饋，當期帳單之享購鈦金卡新增一般消費達NT$3,000~NT$7,999享額外加碼0.2%(即回饋0.7%)，達NT$8,000以上享額外加碼0.5%，即最高1%現金回饋。當期帳單之享購鈦金卡新增一般消費現金回饋比率(不再享紅利積點)消費級距國內消費國外消費NT$1~NT$2,9990.5%1.7%NT$3,000~NT$7,9990.7%>=NT$8,0001.0%注意事項EZTABLE預訂餐廳獨享10%現折持享購鈦金卡於EZTABLE平台(網頁/APP)預付餐費，超過百家餐廳獨享結帳金額現折10%，再享最高5%EZCASH回饋。(每月限量)米其林指南餐廳：MonsieurL、nkụ柴火餐酒館、一號糧倉、點水樓無肉不歡精選：豐舍、教父牛排、肉舖廚房海鮮強棒餐廳：花酒蔵海鮮吧、VGTheSeafoodBar、漢來海港精緻餐酒館：鬍子餐酒、FIFI茶酒沙龍等立即訂位注意事項EZTABLE每週三快閃兩人同行5折優惠持享購鈦金卡於EZTABLE平台(網頁/APP)預訂每月主題精選餐廳，預付EZTABLE優惠餐費享兩人同行5折優惠。(每週快閃限量)立即訂位※每月精選主題適用餐廳以EZTABLE平台公布為準，敬請期待~十月_秋蟹大賞中式火鍋月十一月_到處都能慶感恩節連鎖餐廳十二月_年終聚餐美式餐酒館注意事項台北文華東方酒店『用餐85折起優惠』於台北文華東方酒店指定餐廳刷卡用餐，不分平假日享85折起優惠《Bencotto義大利餐廳》：午晚餐85折《雅閣中餐廳》：午晚餐9折《文華Café》：午晚餐及下午茶85折《M.O.Bar》：85折注意事項Hotels.com刷享購鈦金卡限時優惠最低85折美國指定飯店限時優惠85折108/5/31前，於Hotels.

In [15]:
# driver.find_elements_by_xpath("//div[@class='card-features']/a[@class='link-learn-more']")[2].click()
# catch()
# print(context)
# print(type(context))
# print(len(context))

['新卡滿額禮世界卡2019/7月-2019/10月新卡刷卡禮2019/7/1-2019/10/31期間首次申辦世界卡正卡，核卡日後60天內刷一般消費累積達門檻，享精選好禮回饋:<滿額禮>精緻滿額禮：達NT$15萬以上，「SamsonitePolygon25吋行李箱(藍)」乙個。尊榮滿額禮：達NT$60萬以上，「紅利積點30萬點」。<加碼禮>加碼禮：達NT$18萬以上，「兩次免費機場接或送禮遇」。贈品介紹精緻滿額禮：SamsonitePolygon25吋行李箱(藍)材質：100%PC(聚碳酸酯)尺寸：47X69X31cm淨重：4Kg容量：77L保修期間：3年(相關保修規範以原廠官網說明為準)商品詢問及維修電話:0800-088-349商品簡介：採用FLAT-MAX™結構設計「面蓋與主箱體為20:80比例」，行李箱20%是拉桿系統的面蓋，80%是主要儲存空間的主箱體，打造出特大容量。箱蓋側面有煞車止滑開關，選用日本Hinomoto的EASYBRAKE™煞車輪，拉桿兩側的輪子在煞車系統啟動時，仍可拖行。TSA海關鎖+隱藏式地址貼牌。尊榮滿額禮：紅利積點30萬點紅利積點30萬點可兌換高額航空里程/飯店積分、百貨禮券、商品，讓您輕鬆旅行、購物，盡享寫意自在品味生活。(瞭解更多「舞動紅利」紅利點數兌換辦法)紅利積點30萬點可兌換多樣化優惠：亞洲萬里通74,000里數，或長榮航空60,000哩，或LeClubAccorHotels雅高樂雅會獎勵積分20,000點新光三越百貨商品禮券17,000元，或遠東SOGO百貨商品券17,500元TreeMall紅利線上購物全站超過10萬種品項加碼禮：兩次免費機場接或送禮遇享指定區域至以下機場免費機場接或送禮遇2次。台北松山機場桃園國際機場台中國際機場高雄國際機場(小港機場)24小時服務預約專線(04)2206-1608(持卡人倘符合消費資格，本行將透過簡訊通知，持卡人收到簡訊後才可進行預約。)(本免費機場接送禮遇有限定服務區域，詳細服務適用條件與限制依官網公告為準。)注意事項生日貴賓尊寵禮遇Sisley生日獻禮專屬尊寵生日獻禮：貴賓本人憑世界卡與生日優惠券至全台Sisley專櫃接受專業肌膚諮詢並留下資料，即可兌換生日禮尊寵呵護組。頂級護膚服務現金折抵券NT$2,200：至Sisley指定專櫃或美膚中心預約療程，享NT$2,200療程折抵優

In [ ]:
# # 起始位置設為200，注意螢幕大小，我的螢幕為14吋。
# page = 1
# while page < 5:
#     pos = 200
#     for i in range(10):
#         pos += 220
#         scrolltop(pos)
#         # 注意抓取的值為list，要一個一個取出。
#         driver.find_elements_by_xpath("//div[@class='card-features']/a[@class='link-learn-more']")[i].click()
#         catch()
#         print("現在到第幾頁:", page, "到第幾張卡片:", i + 1)
#         time.sleep(1)
#         if page == 1:
#             driver.back()
#             scrolltop(pos)
#             time.sleep(1)
#         else:
#             driver.back()
#             scrolldown()
#             backpage(page)
#     page += 1
#     backpage(page)
# print("好像沒東西喔!!!")

# context = context[:23]
# print(context)

In [ ]:
# # 起始位置設為200，注意螢幕大小，我的螢幕為14吋。
# page = 1
# context = []
# while page < 5:

#     pos = 200
#     for i in range(10):
#         pos += 220
#         scrolltop(pos)
#         # 注意抓取的值為list，要一個一個取出。
#         driver.find_elements_by_xpath("//div[@class='card-features']/a[@class='link-learn-more']")[i].click()
#         x = catch()
#         x.replace(" ", "")
#         context.append(x)
#         print("現在到第幾頁:", page, "到第幾張卡片:", i + 1)
#         time.sleep(1)
#         if page == 1:
#             driver.back()
#             scrolltop(pos)
#             time.sleep(1)
#         else:
#             driver.back()
#             scrolldown()
#             backpage(page)
#     page += 1
#     backpage(page)

# print("好像沒東西喔!!!")
# context = context[:23]
# print(context)